In [ ]:
# get rid of some chemprop warnings
# import warnings

# warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import os

import pandas as pd

# load the data
df = pd.read_csv('/home/s2861704/ppk/test_kin/prep_storage/K200_EGFR_TEST.csv')

display(df.head())

In [ ]:
# pivot the dataframe to have the accession as columns
df = df.pivot(index="SMILES", columns="accession", values="pchembl_value_Mean")
#df.columns.name = None
df.reset_index(inplace=True)
print(df.columns)

display(df.head())
#Teams message + delete smiles

In [ ]:
# print(len(df))
# max_Nans = len(df) - 30
# print(max_Nans)

In [ ]:
# df_noempty = df
# df_noempty = df_noempty.drop(list(df.columns[df.isna().sum() > max_Nans]), axis=1)
# df = df_noempty.copy()
# display(df)

In [ ]:
column_list = list(df.columns)
print (column_list)

In [ ]:
del column_list[0:1]
print(column_list)

In [ ]:
from qsprpred.data import QSPRDataset
from sklearn.impute import SimpleImputer

target_props = []
for target in column_list:
    target_props.append({'name': target, 'task': "REGRESSION"})

In [ ]:
print(target_props)

In [ ]:
dataset = QSPRDataset(
    name="ChempropMTTestDataset",
    df=df,
    target_props=target_props,
    store_dir="/home/s2861704/ppk/test_kin/test_output/MT/data",
    random_state=42,
    drop_empty=False,
    overwrite=True
)

dataset.getDF()

In [ ]:
from qsprpred.data import QSPRDataset, RandomSplit
from qsprpred.data.descriptors.fingerprints import MorganFP
from qsprpred.data.descriptors.sets import SmilesDesc
from qsprpred.data.sampling.splits import GBMTRandomSplit, GBMTDataSplit

feature_calculators = [SmilesDesc(), MorganFP(radius=3, nBits=2048)]
dataset.prepareDataset(
    split=RandomSplit(test_fraction=0.2, dataset=dataset),
    feature_calculators=feature_calculators,
    recalculate_features=True,
)

dataset.getDF().head()

In [ ]:
from qsprpred.extra.gpu.models.chemprop import ChempropModel

ChempropModel.getAvailableParameters()

In [ ]:
# Create model
os.makedirs("/home/s2861704/ppk/test_kin/test_output/MT/models", exist_ok=True)
model = ChempropModel(
    base_dir='/home/s2861704/ppk/test_kin/test_output/MT/models',
    name='ChempropMTTestModel',
    parameters={"epochs": 5},
    quiet_logger=False
)

In [ ]:
from qsprpred.models import SklearnMetrics, MaskedMetric
from qsprpred.plotting import regression
from qsprpred.tasks import ModelTasks
import numpy as np

r2_score=SklearnMetrics('r2')
masked_metric=MaskedMetric(metric=SklearnMetrics('r2'))

In [ ]:
from qsprpred.models import CrossValAssessor, TestSetAssessor
from qsprpred.models.early_stopping import EarlyStoppingMode

# We can now assess the model performance on the training set using cross validation
CrossValAssessor(masked_metric, split_multitask_scores=True)(model, dataset)

# and on the test set
TestSetAssessor(masked_metric, split_multitask_scores=True)(model, dataset)

# Finally, we need to fit the model on the complete dataset if we want to use it further
# This will save the fitted model and metadata to disk
model.earlyStopping.numEpochs = 20
model.fitDataset(dataset, mode=EarlyStoppingMode.FIXED)

# We can optionally save the model and metadata to disk explicitly as well
_ = model.save()

In [ ]:
import pandas as pd
from sklearn.metrics import explained_variance_score, mean_absolute_error, \
    mean_squared_error, r2_score
import math

# get independent test set
df = pd.read_table(
    "/home/s2861704/ppk/test_kin/test_output/MT/models/ChempropMTTestModel/ChempropMTTestModel.ind.tsv")

# filling NaN-values with fillna() makes the code work, but results in the same issues imputation has. next step to try: individual target plotting
# df = df.fillna(0)

# column names containing original labels or predictions for the tasks
label_names = [i for i in list(df.columns.values) if "Label" in i]
pred_names = [i for i in list(df.columns.values) if "Prediction" in i]

# turn into np array
ylabel = df[label_names].to_numpy()
ypred = df[pred_names].to_numpy()

# get metrics
summary = {}
for metric in [explained_variance_score, mean_absolute_error, mean_squared_error,
               r2_score]:
    score = metric(ylabel, ypred)
    summary[metric.__name__] = score

summary["ModelName"] = model.name

summary

In [ ]:
from plotting_regression_MT import CorrelationPlot

plot = CorrelationPlot([model])
axes, summary = plot.make(save=False, show=True)

In [ ]:
summary